In [5]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.
nH = 8
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()
print(mycc.e_corr)

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Nov 18 15:16:02 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
Default auxbasis cc-pvdz-jkfit is used for H ccpvdz
init E= -3.42956774296905
  HOMO = -0.232078421175356  LUMO = -0.0215923656752635
cycle= 1 E= -4.29143082112552  delta_E= -0.862  |g|= 0.13  |ddm|= 1.52
  HOMO = -0.355602023911344  LUMO = 0.0131748045588607
cycle= 2 E= -4.30345583075545  delta_E= -0.012  |g|= 0.0424  |ddm|= 0.422
  HOMO = -0.351261956784984  LUMO = 0.0360845878432645
cycle= 3 E= -4.30531054292953  delta_E= -0.00185  |g|= 0.0139  |ddm|= 0.21
  HOMO = -0.356668303514654  LUMO = 0.0388386513992613
cycle= 4 E= -4.30552814846736  delta_E= -0.000218  |g|= 0.00252  |ddm|= 0.0889
  HOMO = -0.357134371621119  LUMO = 0.0394645494972193
cycle= 5 E= -4.30553398636939  delta_E= -5.84e-06  |g|= 0.000201  |ddm|= 0.0134
  HOMO = -0.357175260021191  LUMO = 0.0394936567804521
cycle= 6 E= -4.30553401092463  delta_E= -2.46e-08  |g|= 2.37e-05  |ddm|= 0.000614
  HOMO = -0.357177712031946  LUMO = 0.039496796824305

In [6]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [12]:
from pyscf.ci.cisd import CISD
from pyscf.cc.ccsd import CCSD
from ad_afqmc.lno.cc import LNOCCSD
from ad_afqmc.lno_afqmc import lno_maker
from ad_afqmc.lno.base import lno

frozen = nfrozen
thresh = 1e-5
eris = None
run_frg_list = [0]

mfcc = mf

if isinstance(mfcc, (CCSD, CISD)):
    full_cisd = True
    lnomf = mfcc._scf
else:
    full_cisd = False
    lnomf = mfcc

if isinstance(thresh, list):
    thresh_occ, thresh_vir = thresh
else:
    thresh_occ = thresh*10
    thresh_vir = thresh

lno_cc = LNOCCSD(lnomf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = 'boys'
lno_cc.no_type = 'ie'
no_type = 'ie'
lno_cc.frag_lolist = '1o'
lno_cc.force_outcore_ao2mo = True

s1e = lnomf.get_ovlp()
lococc = lno_cc.get_lo(lo_type='boys') # localized active occ orbitals
# lococc,locvir = lno_maker.get_lo(lno_cc,lo_type) ### fix this for DF
if eris is None: eris = lno_cc.ao2mo()

frag_lolist = [[i] for i in range(lococc.shape[1])]
print(frag_lolist)
nfrag = len(frag_lolist)

frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'# lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)

frag_nonvlist = None
if frag_nonvlist is None: frag_nonvlist = lno_cc.frag_nonvlist
if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

eorb_cc = np.empty(nfrag,dtype='float64')
    
from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

for ifrag in run_frg_list:
    print(f'\n########### running fragment {ifrag+1} ##########')

    fraglo = frag_lolist[ifrag]
    orbfragloc = lococc[:,fraglo]
    THRESH_INTERNAL = 1e-10
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    frzfrag, orbfrag, can_orbfrag \
         = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                            THRESH_INTERNAL, thresh_pno,
                            frozen_mask=frozen_mask,
                            frag_target_nocc=None,
                            frag_target_nvir=None,
                            canonicalize=True)

    # mol = mf.mol
    # nocc = mol.nelectron // 2 
    # nao = mol.nao
    # actfrag = np.array([i for i in range(nao) if i not in frzfrag])
    # # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
    # actocc = np.array([i for i in range(nocc) if i in actfrag])
    # actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
    # nactocc = len(actocc)
    # nactocc = len(actocc)
    # nactvir = len(actvir)
    # prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

    # print(f'# active orbitals: {actfrag}')
    # print(f'# active occupied orbitals: {actocc}')
    # print(f'# active virtual orbitals: {actvir}')
    # print(f'# frozen orbitals: {frzfrag}')

    eorb_cc[ifrag],t1,t2 = lno_maker.lno_cc_solver(
                        lnomf,orbfrag,orbfragloc,frozen=frzfrag)


lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228


macro= 1  f(x)= 19.005131246717  delta_f= 19.0051  |g|= 8.24284  1 KF 3 Hx
macro= 2  f(x)= 18.430068362526  delta_f= -0.575063  |g|= 7.35653  1 KF 3 Hx
macro= 3  f(x)= 17.929527568121  delta_f= -0.500541  |g|= 6.49826  1 KF 3 Hx
macro= 4  f(x)= 17.510904508772  delta_f= -0.418623  |g|= 5.54766  1 KF 3 Hx
macro= 5  f(x)= 17.180436592219  delta_f= -0.330468  |g|= 4.51773  1 KF 3 Hx
macro= 6  f(x)= 16.943099158349  delta_f= -0.237337  |g|= 3.42266  1 KF 3 Hx
macro= 7  f(x)= 16.80252356881  delta_f= -0.140576  |g|= 2.27756  1 KF 3 Hx
macro= 8  f(x)= 16.760599364979  delta_f= -0.0419242  |g|= 1.0982  1 KF 3 Hx
macro= 9  f(x)= 16.760598700347  delta_f= -6.64631e-07  |g|= 0.00425071  1 KF 2 Hx
macro= 10  f(x)= 16.760598700348  delta_f= 2.84217e-14  |g|= 2.77074e-10  1 KF 1 Hx
macro X = 10  f(x)= 16.760598700348  |g|= 2.77074e-10  20 intor 10 KF 27 Hx
Lov is saved to /tmp/szsdtkfd
[[0], [1], [2], [3]]
# lno thresh [0.0001, 1e-05]

########### running fragment 1 ##########

WARN: CCSD detected 

In [13]:
print(eorb_cc)
print(sum(eorb_cc))
print(mycc.e_corr)

[-4.67862389e-002  4.94065646e-324  9.88131292e-324  1.48219694e-323]
-0.046786238942967705
-0.1808816224805154


In [ ]:
from functools import partial
from typing import Optional, Union
import h5py
import jax.numpy as jnp
import numpy as np
from pyscf import __config__, mcscf, scf
from pyscf.cc.ccsd import CCSD
from pyscf.cc.uccsd import UCCSD
from ad_afqmc import pyscf_interface

def prep_afqmc(
    mf_cc: Union[scf.uhf.UHF, scf.rhf.RHF, CCSD, UCCSD],
    options: dict,
    basis_coeff: Optional[np.ndarray] = None,
    norb_frozen: int = 0,
    chol_cut: float = 1e-5,
    amp_file = "amplitudes.npz",
    chol_file = "FCIDUMP_chol"
):
    """Prepare AFQMC calculation with mean field trial wavefunction. Writes integrals and mo coefficients to disk.

    Args:
        mf (Union[scf.uhf.UHF, scf.rhf.RHF, mcscf.mc1step.CASSCF]): pyscf mean field object. Used for generating integrals (if not provided) and trial.
        basis_coeff (np.ndarray, optional): Orthonormal basis used for afqmc, given in the basis of ao's. If not provided mo_coeff of mf is used as the basis.
        norb_frozen (int, optional): Number of frozen orbitals. Not supported for custom integrals.
        chol_cut (float, optional): Cholesky decomposition cutoff.
        integrals (dict, optional): Dictionary of integrals in an orthonormal basis, {"h0": enuc, "h1": h1e, "h2": eri}.
    """

    print("#\n# Preparing AFQMC calculation")

    trial = options['trial']

    if isinstance(mf_cc, (CCSD, UCCSD)):
        mf = mf_cc._scf
    else:
        mf = mf_cc

    # if isinstance(mf_or_cc, (CCSD, UCCSD)):
    #     # raise warning about mpi
    #     print(
    #         "# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver."
    #     )
    #     mf = mf_or_cc._scf
    #     cc = mf_or_cc
    #     if cc.frozen is not None:
    #         norb_frozen = cc.frozen
        
    if 'ci' in trial.lower():
        if 'u' in trial.lower():
            ci2aa = t2[0] + 2 * np.einsum("ia,jb->ijab", t1[0], t1[0])
            ci2aa = (ci2aa - ci2aa.transpose(0, 1, 3, 2)) / 2
            ci2aa = ci2aa.transpose(0, 2, 1, 3)
            ci2bb = t2[2] + 2 * np.einsum("ia,jb->ijab", t1[1], t1[1])
            ci2bb = (ci2bb - ci2bb.transpose(0, 1, 3, 2)) / 2
            ci2bb = ci2bb.transpose(0, 2, 1, 3)
            ci2ab = t2[1] + np.einsum("ia,jb->ijab", t1[0], t1[1])
            ci2ab = ci2ab.transpose(0, 2, 1, 3)
            ci1a = np.array(t1[0])
            ci1b = np.array(t1[1])
            np.savez(
                amp_file,
                ci1a=ci1a,
                ci1b=ci1b,
                ci2aa=ci2aa,
                ci2ab=ci2ab,
                ci2bb=ci2bb,
            )
        else:
            ci2 = t2 + np.einsum("ia,jb->ijab", np.array(t1), np.array(t1))
            ci2 = ci2.transpose(0, 2, 1, 3)
            ci1 = np.array(t1)
            np.savez(amp_file, ci1=ci1, ci2=ci2)
    
    elif 'cc' in trial.lower():
        if 'u' in trial.lower():
            t2aa = t2[0]
            t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
            t2aa = t2aa.transpose(0, 2, 1, 3)
            t2bb = t2[2]
            t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
            t2bb = t2bb.transpose(0, 2, 1, 3)
            t2ab = t2[1]
            t2ab = t2ab.transpose(0, 2, 1, 3)
            t1a = np.array(t1[0])
            t1b = np.array(t1[1])
            
            np.savez(
                amp_file,
                t1a=t1a,
                t1b=t1b,
                t2aa=t2aa,
                t2ab=t2ab,
                t2bb=t2bb,
            )
        else:
            t2 = t2
            t2 = t2.transpose(0, 2, 1, 3)
            t1 = np.array(t1)
            np.savez(amp_file, t1=t1, t2=t2)

    mol = mf.mol
    # choose the orbital basis
    if basis_coeff is None:
        if isinstance(mf, scf.uhf.UHF):
            basis_coeff = mf.mo_coeff[0]
        else:
            basis_coeff = mf.mo_coeff

    # calculate cholesky integrals
    print("# Calculating Cholesky integrals")
    
    DFbas = None
    if getattr(mf, "with_df", None) is not None:
        print('# Decomposing ERI with DF')
        DFbas = mf.with_df.auxmol.basis 

    # assert norb_frozen * 2 < sum(
    #     nelec
    # ), "Frozen orbitals exceed number of electrons"
    if 'u' not in trial.lower():
        mc = mcscf.CASSCF(
            mf, mol.nao - norb_frozen, mol.nelectron - 2 * norb_frozen
        )
        nelec = mc.nelecas
        mc.mo_coeff = basis_coeff
        h1e, enuc = mc.get_h1eff()
        _, chol, _, _ = \
            pyscf_interface.generate_integrals(
            mol, mf.get_hcore(), basis_coeff, chol_cut, DFbas=DFbas)
        nao = mf.mol.nao
        chol = chol.reshape((-1, nao, nao))
        chol = chol[:, mc.ncore : mc.ncore + mc.ncas, 
                    mc.ncore : mc.ncore + mc.ncas]
        nbasis = mc.ncas
        print("# Finished calculating Cholesky integrals\n#")
        print("# Size of the correlation space:")
        print(f"# Number of electrons: {nelec}")
        print(f"# Number of basis functions: {nbasis}")
        print(f"# Number of Cholesky vectors: {chol.shape[0]}\n#")
        chol = chol.reshape((-1, nbasis, nbasis))
        v0 = 0.5 * jnp.einsum("nik,njk->ij", chol, chol, optimize="optimal")
        h1e_mod = h1e - v0
        chol = chol.reshape((chol.shape[0], -1))
            
    elif 'u' in trial.lower():
        mc = mcscf.UCASSCF(
            mf, mol.nao - norb_frozen,
            mol.nelectron - 2 * norb_frozen)
        nelec = mc.nelecas
        mc.mo_coeff = mf.mo_coeff
        h1e, enuc = mc.get_h1eff()

        _, chol_a, _, _ = pyscf_interface.generate_integrals(
            mol, mf.get_hcore(), mf.mo_coeff[0], chol_cut, DFbas=DFbas
        )
        # _, chol_b, _, _ = pyscf_interface.generate_integrals(
        #     mol, mf.get_hcore(), mf.mo_coeff[1], chol_cut, DFbas=DFbas
        # )
        nbasis = mc.ncas
        nao = mf.mol.nao
        chol_a = chol_a.reshape((-1, nao, nao))
        # a2b = <B|A>
        # <B|L|B> = <B|A><A|L|A><A|B>
        # if separate build chol might not be the same number
        s1e = mf.get_ovlp()
        a2b = mf.mo_coeff[1].T @ s1e @ mf.mo_coeff[0]
        chol_b = jnp.einsum('pr,grs,sq->gpq',a2b,chol_a,a2b.T)
        chol_a = chol_a[:, mc.ncore[0] : mc.ncore[0] + mc.ncas,
                        mc.ncore[0] : mc.ncore[0] + mc.ncas]
        chol_b = chol_b.reshape((-1, nao, nao))
        chol_b = chol_b[:, mc.ncore[1] : mc.ncore[1] + mc.ncas, 
                        mc.ncore[1] : mc.ncore[1] + mc.ncas]
        v0_a = 0.5 * jnp.einsum("nik,njk->ij", chol_a, chol_a, optimize="optimal")
        v0_b = 0.5 * jnp.einsum("nik,njk->ij", chol_b, chol_b, optimize="optimal")
        h1e = jnp.array(h1e)
        h1e_mod = jnp.array(h1e - jnp.array([v0_a,v0_b]))

        chol = jnp.array(
            [chol_a.reshape(chol_a.shape[0], -1),
             chol_b.reshape(chol_b.shape[0], -1)])
        print("# Finished calculating Cholesky integrals\n#")
        print("# Size of the correlation space:")
        print(f"# Number of electrons: {nelec}")
        print(f"# Number of basis functions: {nbasis}")
        print(f"# Number of Cholesky vectors: {chol_a.shape[0]}\n#")

    
    pyscf_interface.write_dqmc(
        h1e,
        h1e_mod,
        chol,
        sum(nelec),
        nbasis,
        enuc,
        ms=mol.spin,
        filename=chol_file,
    )

In [ ]:
def prep_lnoafqmc_file(mf_cc,mo_coeff,options,norb_act,nelec_act,
                       prjlo=[],norb_frozen=[],ci1=None,ci2=None,
                       use_df_vecs=False,chol_cut=1e-6,
                       option_file='options.bin',
                       mo_file="mo_coeff.npz",
                       amp_file="amplitudes.npz",
                       chol_file="FCIDUMP_chol"):
    
    
    with open(option_file, 'wb') as f:
        pickle.dump(options, f)
    
    # write ccsd amplitudes
    if isinstance(mf_cc, (CCSD, UCCSD)):
        mf = mf_cc._scf
    else:
        mf = mf_cc

    if options["trial"] == "cisd":
        np.savez(amp_file, ci1=ci1, ci2=ci2)

    mol = mf.mol
    # calculate cholesky vectors
    h1e, chol, nelec, enuc, nbasis, nchol = [ None ] * 6
    print('# Generating Cholesky Integrals')

    mc = mcscf.CASSCF(mf, norb_act, nelec_act)
    mc.frozen = norb_frozen
    nelec = mc.nelecas
    mc.mo_coeff = mo_coeff
    h1e, enuc = mc.get_h1eff()

    nbasis = h1e.shape[-1]
    if isinstance(norb_frozen, (int, float)) and norb_frozen == 0:
        norb_frozen = []
    elif isinstance(norb_frozen, int):
        norb_frozen = np.arange(norb_frozen)
    print(f'# frozen orbitals: {norb_frozen}')
    act = np.array([i for i in range(mol.nao) if i not in norb_frozen])
    print(f'# local active orbitals: {act}') #yichi
    print(f'# local active space size: {len(act)}') #yichi

    if getattr(mf, "with_df", None) is not None:
        if use_df_vecs:
            print('# use DF vectors as Cholesky vectors')
            _, chol, _, _ = pyscf_interface.generate_integrals(
                mol,mf.get_hcore(),mo_coeff[:,act],DFbas=mf.with_df.auxmol.basis)
        else:
            # using DF basis chol is inefficient for LNO
            print("# composing AO ERIs from DF basis")
            from pyscf import df
            chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
            chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
            chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
            eri_ao_df = lib.einsum('lpq,lrs->pqrs', chol_df, chol_df)
            # eri_df.reshape((nao*nao, nao*nao))
            print("# decomposing MO ERIs to Cholesky vectors")
            print(f"# Cholesky cutoff is: {chol_cut}")
            eri_mo_df = ao2mo.kernel(eri_ao_df,mo_coeff[:,act],compact=False)
            eri_mo_df = eri_mo_df.reshape(nbasis**2,nbasis**2)
            chol = pyscf_interface.modified_cholesky(eri_mo_df,max_error=chol_cut)
    else:
        eri_mo = ao2mo.kernel(mf.mol,mo_coeff[:,act],compact=False)
        chol = pyscf_interface.modified_cholesky(eri_mo,max_error=chol_cut)
    
    print("# Finished calculating Cholesky integrals\n")
    print('# Size of the correlation space')
    print(f'# Number of electrons: {nelec}')
    print(f'# Number of basis functions: {nbasis}')
    print(f'# Number of Cholesky vectors: {chol.shape[0]}\n')
    
    chol = chol.reshape((-1, nbasis, nbasis))
    v0 = 0.5 * lib.einsum('nik,njk->ij', chol, chol, optimize='optimal')
    h1e_mod = h1e - v0
    chol = chol.reshape((chol.shape[0], -1))

    # write mo coefficients
    trial_coeffs = np.empty((2, nbasis, nbasis))
    # overlap = mf.get_ovlp(mol)
    if isinstance(mf, (scf.uhf.UHF, scf.rohf.ROHF)):
        uhfCoeffs = np.empty((nbasis, 2 * nbasis))
        overlap = mf.get_ovlp(mol)
        if isinstance(mf, scf.uhf.UHF):
            q, r = np.linalg.qr(mo_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[0][:, norb_frozen:]))
            uhfCoeffs[:, :nbasis] = q
            q, r = np.linalg.qr(mo_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[1][:, norb_frozen:]))
            uhfCoeffs[:, nbasis:] = q
        else:
            q, r = np.linalg.qr(mo_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[:, norb_frozen:]))
            uhfCoeffs[:, :nbasis] = q
            uhfCoeffs[:, nbasis:] = q

        trial_coeffs[0] = uhfCoeffs[:, :nbasis]
        trial_coeffs[1] = uhfCoeffs[:, nbasis:]
        np.savez(mo_file,mo_coeff=trial_coeffs,prjlo=prjlo)

    elif isinstance(mf, scf.rhf.RHF):
        #q, r = np.linalg.qr(mo_coeff[:, norb_frozen:].T.dot(overlap).dot(mf.mo_coeff[:, norb_frozen:]))
        q = np.eye(mol.nao- len(norb_frozen)) # in mo basis
        trial_coeffs[0] = q
        trial_coeffs[1] = q
        np.savez(mo_file,mo_coeff=trial_coeffs,prjlo=prjlo)

    write_dqmc(mf.e_tot,h1e,h1e_mod,chol,sum(nelec),nbasis,enuc,ms=mol.spin,
               filename=chol_file,mo_coeffs=trial_coeffs)
    
    return None